In [4]:
import pandas as pd
import numpy as np
import random

In [2]:
def new_num_iter(num_i):
    if num_i <= 10:
        new_num = [10, 50, 100]
    elif num_i <= 100:
        new_num = [50, 100, 150, 200]
    elif num_i <= 200:
        new_num = [150, 200, 300]
    else:
        new_num = [num_i - 100, num_i, num_i + 100]
    return new_num

def new_lr(lr):
    
    def count_zero(x):
        if x < 1:
            iter = 1
            while True:
                if x * 10**iter >= 1:
                    break
                iter = iter+1
            return iter
        elif x >= 1:
            iter = 1
            while True:
                if x / 10**iter < 1:
                    break
                iter = iter+1
            return -iter
    
    def back_step(lr, lr_step):
        if lr - lr_step*2 <=0 :
            tenth = 10**(count_zero(lr)+1)
            return int(lr*tenth/3)/tenth, int(lr*tenth/2)/tenth
        else:
            return lr_step, lr_step*2
    
    if lr <= 0.001:
        lr_step = 0.0002
        lr_step_m1, lr_step_m2 = back_step(lr, lr_step)
    elif lr <= 0.01:
        lr_step = 0.002
        lr_step_m1, lr_step_m2 = back_step(lr, lr_step)
    elif lr <= 0.1:
        lr_step = 0.02
        lr_step_m1, lr_step_m2 = back_step(lr, lr_step)
    elif lr <= 1:
        lr_step = 0.2
        lr_step_m1, lr_step_m2 = back_step(lr, lr_step)
    elif lr <= 10:
        lr_step = 2
        lr_step_m1, lr_step_m2 = back_step(lr, lr_step)
    else:
        lr_step = 20
        lr_step_m1, lr_step_m2 = back_step(lr, lr_step)

    return [lr-lr_step_m2, lr-lr_step_m1, lr, lr+lr_step, lr+lr_step*2]

def new_depth(depth):
    if depth == None:
        new_depth = [None, 1, 2, 3, 4]
        new_depth = [1, 2, 3, 4]
    elif depth <= 2:
        new_depth = [1, 2, 3, 4]
    elif depth <= 4:
        new_depth = [3, 4, 5, 6]
    elif depth <= 6:
        new_depth = [5, 6, 7, 8]
    else:
        new_depth = [7, 8, 9, 10, 11]
    return new_depth

def new_leaves(leaves):
    if leaves <= 3:
        new_leaves = [2, 3, 4]
    elif leaves <= 10:
        new_leaves = [leaves-2, leaves-1, leaves, leaves+1, leaves+2]
    elif leaves <= 20:
        new_leaves = [leaves-4, leaves-2, leaves, leaves+2, leaves+4]
    else:
        new_leaves = [leaves-8, leaves-4, leaves, leaves+4, leaves+8]
    return new_leaves

def new_subsample(subsam):
    if subsam == 0.002:
        return [0.0005, 0.001, 0.002, 0.003]
    elif subsam <= 0.01:
        step = 0.001
    elif subsam <= 0.1:
        step = 0.01
    elif subsam <= 1:
        step = 0.2
        
    return [subsam-step, subsam, subsam+step]
    
def new_nestimator(nest):
    if nest <= 10:
        return [5, 10, 30]
    elif nest <= 50:
        return [10, 30, 50, 70]
    elif nest <= 100:
        return [60, 80, 100, 120]
    elif nest <= 140:
        return [100, 120, 140, 160]
    else :
        return [140, 160, 190, 220]
    
def make_new_parameter(params_o, df_concat):
    param_new = {}
    param_new["boosting_type"] = ['gbdt']
    
    num_i = df_concat.loc["num_iterations"].iloc[0]
    param_new["num_iterations"] = new_num_iter(num_i) #새로운 값을 중간에 배치, 100단위로 순서
    
    lr = df_concat.loc["learning_rate"].iloc[0]
    param_new["learning_rate"] = new_lr(lr)
    
    depth = df_concat.loc["max_depth"].iloc[0]
    param_new["max_depth"] = new_depth(depth) # 가장 먼저 튜닝 필요 -1이 default (무한깊이) 일반적으로 default가 가장 좋음.
    
    leaves = df_concat.loc["num_leaves"].iloc[0]  
    param_new["num_leaves"] = new_leaves(leaves) #  # 두번째로 중요, num_leaves = 2^max_depth보다 작아야 함. 데이터가 적으면 작은 숫자로
    
    subsam = df_concat.loc["subsample"].iloc[0]  
    param_new["subsample"] = new_subsample(subsam) # 세번째로 중요. = bagging fraction, row sampling. 아래 colsample_bytree과 같이 튜닝.
    
    param_new["colsample_bytree"] = [0.25, 0.3, 0.35, 0.4, 0.45] # = feature fraction, column sampling. 위의 subsample과 같이 튜닝.
    param_new["objective"] = ['binary']
    param_new["metric"] = ['binary_logloss'] # "metric": ['loss'], # mae : mean absolute error, mse : mean squared error, 
                      # binary_logloss : loss for binary classification, multi_logloss : loss for multi classification

    nest = df_concat.loc["n_estimators"].iloc[0] 
    param_new["n_estimators"] = new_nestimator(nest) # [None, 3, 4, 5]
    param_new["max_delta_step"] = [0.5, 0.6, 0.7, 0.8, 0.9] # best value found, default = 0
    param_new["scale_pos_weight"] = [3, 10, 20] #  class imbalance 경감, scale_pos_weight > 0.0, defaut
#     param_new["is_unbalance"] = [1],  # 불균형 셋 조정, 주의: "scale_pos_weight" 동시에 사용할 수 없음.
    
#     param_new["scoring"] = ['accuracy', 'precision'],
#     param_new["early_stopping_rounds"] = [50],

    
    print("-------- End of Parameter Setting -------")
    
    return param_new

In [3]:
def new_num_iter(num_i):
    if num_i <= 10:
        new_num = [10, 50, 100]
    elif num_i <= 100:
        new_num = [50, 100, 150, 200]
    elif num_i <= 200:
        new_num = [150, 200, 300]
    else:
        new_num = [num_i - 100, num_i, num_i + 100]
    return new_num

for i in range(0, 500, 1):
    a = new_num_iter(i)
    print(i, a)
        

0 [10, 50, 100]
1 [10, 50, 100]
2 [10, 50, 100]
3 [10, 50, 100]
4 [10, 50, 100]
5 [10, 50, 100]
6 [10, 50, 100]
7 [10, 50, 100]
8 [10, 50, 100]
9 [10, 50, 100]
10 [10, 50, 100]
11 [50, 100, 150, 200]
12 [50, 100, 150, 200]
13 [50, 100, 150, 200]
14 [50, 100, 150, 200]
15 [50, 100, 150, 200]
16 [50, 100, 150, 200]
17 [50, 100, 150, 200]
18 [50, 100, 150, 200]
19 [50, 100, 150, 200]
20 [50, 100, 150, 200]
21 [50, 100, 150, 200]
22 [50, 100, 150, 200]
23 [50, 100, 150, 200]
24 [50, 100, 150, 200]
25 [50, 100, 150, 200]
26 [50, 100, 150, 200]
27 [50, 100, 150, 200]
28 [50, 100, 150, 200]
29 [50, 100, 150, 200]
30 [50, 100, 150, 200]
31 [50, 100, 150, 200]
32 [50, 100, 150, 200]
33 [50, 100, 150, 200]
34 [50, 100, 150, 200]
35 [50, 100, 150, 200]
36 [50, 100, 150, 200]
37 [50, 100, 150, 200]
38 [50, 100, 150, 200]
39 [50, 100, 150, 200]
40 [50, 100, 150, 200]
41 [50, 100, 150, 200]
42 [50, 100, 150, 200]
43 [50, 100, 150, 200]
44 [50, 100, 150, 200]
45 [50, 100, 150, 200]
46 [50, 100, 150, 

In [ ]:
params_o = {
    "boosting_type" : ['gbdt'], # ['dart', 'goss'], # dart : 신경망의 드롭아웃을 적용시킨 방법, 
    "num_iterations" : [300], #[450, 500, 550], #"num_iterations" 
    "learning_rate": [0.03, 0.04, 0.051, 0.06, 0.07], 
    "max_depth": [-1, 2, 3, 4], # 가장 먼저 튜닝 필요 -1이 default (무한깊이) 일반적으로 default가 가장 좋음.
    "num_leaves": [2, 3, 4, 5, 6], #  # 두번째로 중요, num_leaves는 작은 데이터면 작은 숫자로
    "subsample": [1, 0.001, 0.002],# 세번째로 중요. = bagging fraction, row sampling. 아래 colsample_bytree과 같이 튜닝.
    "colsample_bytree": [0.3, 0.35, 0.4, 0.45, 0.5], # = feature fraction, column sampling. 위의 subsample과 같이 튜닝.
    "objective": ['binary'],
    "metric": ['binary_logloss'], # "metric": ['loss'], # mae : mean absolute error, mse : mean squared error, 
                      # binary_logloss : loss for binary classification, multi_logloss : loss for multi classification

    "n_estimators": [None, 3, 4, 5],
    "max_delta_step": [0.4, 0.5, 0.6, 0.7, 0.8], # best value found, default = 0
    "scale_pos_weight": [2, 10, 15], # class imb

In [25]:
lr = (np.logspace(-3.5, 1, 30)*10000).astype(int)
lr

array([     3,      4,      6,      9,     13,     18,     26,     38,
           55,     78,    112,    161,    230,    329,    470,    672,
          961,   1373,   1963,   2807,   4012,   5736,   8199,  11721,
        16754,  23950,  34235,  48939,  69956, 100000])

In [47]:
s = 1
sa =[s]
for i in range(170):
    s = s*(1+ 0.05)
    sa.append(s)

In [54]:
bb = np.trunc(sa*10000)/10000

In [64]:
bb[1]

0.0000

In [18]:
import pandas as pd
import numpy as np
from IPython.display import display
pd.options.display.float_format = '{:,.5f}'.format

In [6]:
import random

In [23]:
item = [5, 10, 20, 30, 50, 70] 
item = [0.01, 0.02, 0.04, 0.045, 0.05] 
int3 = (max(item) - min(item))/len(item)
int3

0.008

In [28]:
sel = random.choice(item)
nex = [sel-int3, sel, sel+int3]
nex

[0.032, 0.04, 0.048]

In [5]:
def make_new_param(item, sel, int_flo):
    item = sorted(item)
    interval = (max(item) - min(item))/len(item)
    if (interval < 0.00001):
        cut_r = 6
    elif (interval < 0.0001):
        cut_r = 5
    elif (interval < 0.001):
        cut_r = 4
    elif (interval < 0.01):
        cut_r = 3
    elif (interval < 0.1):
        cut_r = 2
    elif (interval < 1):
        cut_r = 1
    else:
        cut_r = 0
    
    interval = round(((max(item) - min(item))/len(item)), cut_r)
    if interval < sel*0.1 :  # 원소값의 10%미만일 경우는 10%로 interval 설정
        interval = sel * 0.1
    loc = item.index(sel) # 선정된 위치에 따라 다음 리스트 구성 기준 차이 
    # 가장 큰 쪽의 요소가 선정되었을 경우, sel, [sel+interval/2, sel+interval*2, sel+interval*3]
    # 가장 작은 쪽의 요소가 선정되었을 경우, sel-interval*3, sel-interval*2, sel-interval/2]
    #              if sel-interval*3 < 0 : [sel*0.25, sel*0.5, sel*0.75]
    # 가장 가운데 요소가 선정되었을 경우, 세분화 시킴 [ sel-int*0.5, sel, sel+int*0.5]
    print("sel", sel)
    if loc == len(item)-1 : #가장 큰 쪽의 요소가 선정
        new_param = [sel-interval*0.5, sel+interval*0.5, sel+interval*2, sel+interval*3, sel+interval*4]
    elif loc == 0: # 가장 작은 쪽의 요소가 선정
        if (sel-interval*4) <= 0 :
            new_param = [sel*0.2, sel*0.4, sel*0.6, sel*0.8, sel*1.2]
        else:
            new_param = [sel-interval*4, sel-interval*3, sel-interval*2, sel-interval*0.5, sel+interval*0.5]
    else:
        if (sel-interval) <= 0 :
            new_param = [sel-interval*0.9, sel-interval*0.5, sel+interval*0.5, sel+interval]
        else:
            new_param = [sel-interval, sel-interval*0.5, sel+interval*0.5, sel+interval]

    if int_flo == 0: # 0: integer, 1: flaot
        setp = np.int32(np.rint(new_param))
        new_list_nonzero = [i for i in list(set(setp)) if i != 0] # 0이 있으면 삭제
        if len(new_list_nonzero) <= 2 :
            new_list_nonzero = [1, 2, 3, 4]
        return sorted(new_list_nonzero)
        
    return list(np.round(new_param, 5))


In [18]:
item = [5, 10, 20] 
item = [10, 30, 50, 70]
# item = [60, 80, 100, 120]
# item = [0.01, 0.02, 0.04, 0.045, 0.05] 
print("**0** ", item)
for i in range(20):
    sel = random.choice(item)
    n = make_new_param(item, sel, 0)
    print(f'**{i}** {n}')
    item = n

**0**  [10, 30, 50, 70]
sel 50
**0** [35, 42, 58, 65]
sel 42
**1** [34, 38, 46, 50]
sel 34
**2** [18, 22, 26, 32, 36]
sel 26
**3** [22, 24, 28, 30]
sel 24
**4** [22, 23, 25, 26]
sel 25
**5** [22, 24, 26, 28]
sel 24
**6** [22, 23, 25, 26]
sel 22
**7** [13, 15, 18, 21, 23]
sel 13
**8** [5, 7, 9, 12, 14]
sel 12
**9** [10, 11, 13, 14]
sel 11
**10** [1, 2, 3, 4]
sel 4
**11** [4, 6, 7]
sel 6
**12** [5, 6, 7]
sel 7
**13** [7, 8, 9, 10]
sel 8
**14** [7, 8, 9]
sel 9
**15** [9, 11, 12, 13]
sel 9
**16** [5, 6, 7, 8, 10]
sel 10
**17** [10, 12, 13, 14]
sel 13
**18** [1, 2, 3, 4]
sel 1
**19** [1, 2, 3, 4]


In [17]:
round(1111.23456, 1)

1111.2

In [108]:
def decimal_p (x):
    if (x//10) <= 9:
        cut_d = 0
    elif (x//100) <= 9:
        cut_d = 1
    elif (x//1000) <= 9:
        cut_d = 2
    else:
        cut_d = 3
    return cut_d

In [109]:
decimal_p(19)

0

In [171]:
new_param = [30.23488, 30.85904, 34.10736, 35.73152]

In [134]:
(np.int32(aa)))))

[34, 35, 30]

In [175]:
setp = np.int32(np.rint(new_param))
new_list_nonzero = [i for i in list(set(setp)) if i != 0] # 0이 있으면 삭제
new_list_nonzero = new_list_nonzero.sort()
new_list_nonzero

In [176]:
[i for i in list(set(setp)) if i != 0]

[34, 36, 30, 31]

In [182]:
n = sorted([4,2,5,3])
n

[2, 3, 4, 5]